In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GroupKFold
from sklearn.utils import shuffle
from six.moves import cPickle

In [2]:
X_train = pd.read_hdf('X_train_w32.h5')
y_train = pd.read_hdf('y_train_w32.h5')
X_test = pd.read_hdf('X_test_w32.h5')
X_test.drop('reordered', axis=1, inplace=True)

In [3]:
#shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=1969)
#add group K fold
K = 5

In [4]:
groups = X_train.user_id
group_kfold = GroupKFold(n_splits=K)
group_kfold.get_n_splits(X_train, y_train, groups)
myGroups = list(group_kfold.split(X_train, y_train, groups))

In [5]:
X_train.drop(['eval_set', 'user_id', 'product_id', 'order_id'], axis=1, inplace=True)

In [6]:
train = lgb.Dataset(data=X_train, label=y_train)

In [7]:
params = {'task': 'train',
          'boosting': 'gbdt',
          'application':'binary',
          'learning_rate':0.1,
          'objective': 'binary',
          'metric': {'binary_logloss', 'auc'},
          'num_leaves': 128, #96, 128, 192, 256, 512
          'max_depth': 10,
          'num_threads':4,
          'device':'cpu',
          'feature_fraction': 0.75, #0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65
          'bagging_fraction': 0.95, #0.95, 0.9, 0.85, 0.8
          'bagging_freq': 5,
          'lambda_l1':0.01,
          'lambda_l2':0.01,
          'is_unbalance':False,
          'verbosity':-1}

In [8]:
#grid search
max_depth = [10]
num_leaves = [128]
feature_fraction = [0.75]
bagging_fraction = [0.8, 0.7, 0.6, 0.5]
lambda_l1 = [0.1, 0.3, 1, 3, 10, 30]
lambda_l2 = [0.3, 1, 3, 10, 30]
results = list()
for md in max_depth:
    params['max_depth'] = md
    for nl in num_leaves:
        params['num_leaves'] = nl
        for ff in feature_fraction:
            params['feature_fraction'] = ff
            for bf in bagging_fraction:
                params['bagging_fraction'] = bf
                for l1 in lambda_l1:
                    params['lambda_l1'] = l1
                    for l2 in lambda_l2:
                        params['lambda_l2'] = l2
                        param = [md, nl, ff, bf, l1, l2]
                        print(param)
                        bst = lgb.cv(params, train, num_boost_round=5000, metrics={'binary_logloss'}, folds=myGroups,
                                     early_stopping_rounds=20, verbose_eval=True, seed=777)
                        results.append((param, bst))

[10, 128, 0.75, 0.8, 0.1, 0.3]
[1]	cv_agg's binary_logloss: 0.625382 + 7.14017e-05
[2]	cv_agg's binary_logloss: 0.570013 + 0.000135615
[3]	cv_agg's binary_logloss: 0.524026 + 0.000190171
[4]	cv_agg's binary_logloss: 0.485428 + 0.000245714
[5]	cv_agg's binary_logloss: 0.452768 + 0.000288156
[6]	cv_agg's binary_logloss: 0.424938 + 0.000330235
[7]	cv_agg's binary_logloss: 0.401144 + 0.000371812
[8]	cv_agg's binary_logloss: 0.380635 + 0.000408837
[9]	cv_agg's binary_logloss: 0.362959 + 0.000436527
[10]	cv_agg's binary_logloss: 0.347627 + 0.000467731
[11]	cv_agg's binary_logloss: 0.334332 + 0.000503579
[12]	cv_agg's binary_logloss: 0.322782 + 0.000526028
[13]	cv_agg's binary_logloss: 0.312749 + 0.000550792
[14]	cv_agg's binary_logloss: 0.304072 + 0.000575473
[15]	cv_agg's binary_logloss: 0.296462 + 0.000597567
[16]	cv_agg's binary_logloss: 0.289857 + 0.00061856
[17]	cv_agg's binary_logloss: 0.284066 + 0.000629439
[18]	cv_agg's binary_logloss: 0.279021 + 0.000656015
[19]	cv_agg's binary_logl

[155]	cv_agg's binary_logloss: 0.24338 + 0.000829172
[156]	cv_agg's binary_logloss: 0.24338 + 0.000828073
[157]	cv_agg's binary_logloss: 0.243379 + 0.000827915
[158]	cv_agg's binary_logloss: 0.24338 + 0.000827386
[159]	cv_agg's binary_logloss: 0.243379 + 0.000828002
[160]	cv_agg's binary_logloss: 0.24338 + 0.000825897
[161]	cv_agg's binary_logloss: 0.24338 + 0.000824739
[162]	cv_agg's binary_logloss: 0.243378 + 0.000821346
[163]	cv_agg's binary_logloss: 0.243379 + 0.000820704
[164]	cv_agg's binary_logloss: 0.243378 + 0.000820773
[165]	cv_agg's binary_logloss: 0.243379 + 0.000820663
[166]	cv_agg's binary_logloss: 0.243377 + 0.000820125
[167]	cv_agg's binary_logloss: 0.243378 + 0.000819591
[168]	cv_agg's binary_logloss: 0.243377 + 0.000819491
[169]	cv_agg's binary_logloss: 0.243377 + 0.000818729
[170]	cv_agg's binary_logloss: 0.243377 + 0.000818987
[171]	cv_agg's binary_logloss: 0.243379 + 0.00081928
[172]	cv_agg's binary_logloss: 0.24338 + 0.000819707
[173]	cv_agg's binary_logloss: 0.24

[114]	cv_agg's binary_logloss: 0.24348 + 0.000820792
[115]	cv_agg's binary_logloss: 0.243473 + 0.000825456
[116]	cv_agg's binary_logloss: 0.243469 + 0.000822794
[117]	cv_agg's binary_logloss: 0.243464 + 0.000823439
[118]	cv_agg's binary_logloss: 0.243459 + 0.000823804
[119]	cv_agg's binary_logloss: 0.243456 + 0.000824565
[120]	cv_agg's binary_logloss: 0.243452 + 0.000820214
[121]	cv_agg's binary_logloss: 0.243449 + 0.00082113
[122]	cv_agg's binary_logloss: 0.243445 + 0.000821116
[123]	cv_agg's binary_logloss: 0.243437 + 0.00082062
[124]	cv_agg's binary_logloss: 0.243436 + 0.000821927
[125]	cv_agg's binary_logloss: 0.243431 + 0.000822382
[126]	cv_agg's binary_logloss: 0.243421 + 0.000818837
[127]	cv_agg's binary_logloss: 0.243418 + 0.000818465
[128]	cv_agg's binary_logloss: 0.243418 + 0.00081707
[129]	cv_agg's binary_logloss: 0.243418 + 0.000816773
[130]	cv_agg's binary_logloss: 0.243419 + 0.000815925
[131]	cv_agg's binary_logloss: 0.243417 + 0.000816158
[132]	cv_agg's binary_logloss: 0

[73]	cv_agg's binary_logloss: 0.2438 + 0.000841523
[74]	cv_agg's binary_logloss: 0.243783 + 0.000841066
[75]	cv_agg's binary_logloss: 0.243765 + 0.000841237
[76]	cv_agg's binary_logloss: 0.243747 + 0.000837948
[77]	cv_agg's binary_logloss: 0.24373 + 0.000839541
[78]	cv_agg's binary_logloss: 0.243711 + 0.000834579
[79]	cv_agg's binary_logloss: 0.243701 + 0.000833095
[80]	cv_agg's binary_logloss: 0.243689 + 0.000835861
[81]	cv_agg's binary_logloss: 0.243678 + 0.000836071
[82]	cv_agg's binary_logloss: 0.243664 + 0.000835457
[83]	cv_agg's binary_logloss: 0.243658 + 0.000835412
[84]	cv_agg's binary_logloss: 0.243651 + 0.000835824
[85]	cv_agg's binary_logloss: 0.243642 + 0.000833377
[86]	cv_agg's binary_logloss: 0.243626 + 0.000833749
[87]	cv_agg's binary_logloss: 0.243611 + 0.000835832
[88]	cv_agg's binary_logloss: 0.243603 + 0.000835412
[89]	cv_agg's binary_logloss: 0.243595 + 0.000836928
[90]	cv_agg's binary_logloss: 0.243587 + 0.000839257
[91]	cv_agg's binary_logloss: 0.24358 + 0.0008379

[15]	cv_agg's binary_logloss: 0.296549 + 0.000598624
[16]	cv_agg's binary_logloss: 0.289945 + 0.000620626
[17]	cv_agg's binary_logloss: 0.284151 + 0.000631407
[18]	cv_agg's binary_logloss: 0.279102 + 0.000653172
[19]	cv_agg's binary_logloss: 0.274677 + 0.000666701
[20]	cv_agg's binary_logloss: 0.270825 + 0.000678896
[21]	cv_agg's binary_logloss: 0.267466 + 0.000697495
[22]	cv_agg's binary_logloss: 0.264561 + 0.000708471
[23]	cv_agg's binary_logloss: 0.261992 + 0.000710221
[24]	cv_agg's binary_logloss: 0.259752 + 0.000716363
[25]	cv_agg's binary_logloss: 0.257807 + 0.000727147
[26]	cv_agg's binary_logloss: 0.256114 + 0.000738118
[27]	cv_agg's binary_logloss: 0.254636 + 0.000744746
[28]	cv_agg's binary_logloss: 0.253329 + 0.000755471
[29]	cv_agg's binary_logloss: 0.252204 + 0.000766088
[30]	cv_agg's binary_logloss: 0.251212 + 0.000774048
[31]	cv_agg's binary_logloss: 0.250346 + 0.000781215
[32]	cv_agg's binary_logloss: 0.249577 + 0.000786015
[33]	cv_agg's binary_logloss: 0.248925 + 0.000

...

[92]	cv_agg's binary_logloss: 0.243779 + 0.000813526
[93]	cv_agg's binary_logloss: 0.243772 + 0.000812229
[94]	cv_agg's binary_logloss: 0.243765 + 0.000817047
[95]	cv_agg's binary_logloss: 0.243756 + 0.000816905
[96]	cv_agg's binary_logloss: 0.243745 + 0.000818773
[97]	cv_agg's binary_logloss: 0.243735 + 0.000823548
[98]	cv_agg's binary_logloss: 0.243721 + 0.000825092
[99]	cv_agg's binary_logloss: 0.243713 + 0.000825097
[100]	cv_agg's binary_logloss: 0.243709 + 0.000825862
[101]	cv_agg's binary_logloss: 0.243705 + 0.000824313
[102]	cv_agg's binary_logloss: 0.243699 + 0.000823505
[103]	cv_agg's binary_logloss: 0.243692 + 0.000823371
[104]	cv_agg's binary_logloss: 0.243685 + 0.000821325
[105]	cv_agg's binary_logloss: 0.243679 + 0.000820323
[106]	cv_agg's binary_logloss: 0.243672 + 0.000821314
[107]	cv_agg's binary_logloss: 0.243664 + 0.000820861
[108]	cv_agg's binary_logloss: 0.243658 + 0.000819079
[109]	cv_agg's binary_logloss: 0.243653 + 0.000818953
[110]	cv_agg's binary_logloss: 0.243

[245]	cv_agg's binary_logloss: 0.243345 + 0.000806987
[246]	cv_agg's binary_logloss: 0.243344 + 0.000807966
[247]	cv_agg's binary_logloss: 0.243344 + 0.000808489
[248]	cv_agg's binary_logloss: 0.243345 + 0.000808578
[249]	cv_agg's binary_logloss: 0.243344 + 0.00080794
[250]	cv_agg's binary_logloss: 0.243344 + 0.000807831
[251]	cv_agg's binary_logloss: 0.243344 + 0.000807434
[252]	cv_agg's binary_logloss: 0.243343 + 0.000807777
[253]	cv_agg's binary_logloss: 0.243344 + 0.00080716
[254]	cv_agg's binary_logloss: 0.243343 + 0.000804645
[255]	cv_agg's binary_logloss: 0.243343 + 0.00080325
[256]	cv_agg's binary_logloss: 0.243343 + 0.00080441
[257]	cv_agg's binary_logloss: 0.243343 + 0.000805629
[258]	cv_agg's binary_logloss: 0.243342 + 0.00080653
[259]	cv_agg's binary_logloss: 0.243343 + 0.000807049
[260]	cv_agg's binary_logloss: 0.243343 + 0.000808678
[261]	cv_agg's binary_logloss: 0.243343 + 0.00080875
[262]	cv_agg's binary_logloss: 0.243342 + 0.000808849
[263]	cv_agg's binary_logloss: 0.2

[117]	cv_agg's binary_logloss: 0.243611 + 0.000821908
[118]	cv_agg's binary_logloss: 0.243602 + 0.000821528
[119]	cv_agg's binary_logloss: 0.243599 + 0.000822224
[120]	cv_agg's binary_logloss: 0.243595 + 0.000821175
[121]	cv_agg's binary_logloss: 0.243592 + 0.000822055
[122]	cv_agg's binary_logloss: 0.243586 + 0.000822077
[123]	cv_agg's binary_logloss: 0.243583 + 0.000822519
[124]	cv_agg's binary_logloss: 0.243582 + 0.000821432
[125]	cv_agg's binary_logloss: 0.243578 + 0.000821907
[126]	cv_agg's binary_logloss: 0.243565 + 0.000819613
[127]	cv_agg's binary_logloss: 0.243558 + 0.000818554
[128]	cv_agg's binary_logloss: 0.243557 + 0.000818964
[129]	cv_agg's binary_logloss: 0.243555 + 0.00081864
[130]	cv_agg's binary_logloss: 0.243553 + 0.000817226
[131]	cv_agg's binary_logloss: 0.24355 + 0.000816999
[132]	cv_agg's binary_logloss: 0.243541 + 0.000816958
[133]	cv_agg's binary_logloss: 0.243539 + 0.000816022
[134]	cv_agg's binary_logloss: 0.243529 + 0.000814247
[135]	cv_agg's binary_logloss:

[2]	cv_agg's binary_logloss: 0.57036 + 0.000113137
[3]	cv_agg's binary_logloss: 0.524499 + 0.00016696
[4]	cv_agg's binary_logloss: 0.48599 + 0.000210179
[5]	cv_agg's binary_logloss: 0.453413 + 0.000258662
[6]	cv_agg's binary_logloss: 0.425638 + 0.000314256
[7]	cv_agg's binary_logloss: 0.401897 + 0.00035903
[8]	cv_agg's binary_logloss: 0.381442 + 0.000396023
[9]	cv_agg's binary_logloss: 0.363805 + 0.000428353
[10]	cv_agg's binary_logloss: 0.348498 + 0.000462646
[11]	cv_agg's binary_logloss: 0.335236 + 0.000484553
[12]	cv_agg's binary_logloss: 0.323696 + 0.000511059
[13]	cv_agg's binary_logloss: 0.313673 + 0.00053504
[14]	cv_agg's binary_logloss: 0.305029 + 0.000565858
[15]	cv_agg's binary_logloss: 0.297417 + 0.00059261
[16]	cv_agg's binary_logloss: 0.290807 + 0.000614386
[17]	cv_agg's binary_logloss: 0.285001 + 0.000628483
[18]	cv_agg's binary_logloss: 0.279932 + 0.000649741
[19]	cv_agg's binary_logloss: 0.275504 + 0.000665649
[20]	cv_agg's binary_logloss: 0.271639 + 0.000682779
[21]	cv

[157]	cv_agg's binary_logloss: 0.243473 + 0.000817909
[158]	cv_agg's binary_logloss: 0.243473 + 0.000818835
[159]	cv_agg's binary_logloss: 0.243471 + 0.000819512
[160]	cv_agg's binary_logloss: 0.24347 + 0.000819166
[161]	cv_agg's binary_logloss: 0.243467 + 0.00081621
[162]	cv_agg's binary_logloss: 0.243463 + 0.000814057
[163]	cv_agg's binary_logloss: 0.243462 + 0.000813046
[164]	cv_agg's binary_logloss: 0.243458 + 0.000811761
[165]	cv_agg's binary_logloss: 0.243458 + 0.000812935
[166]	cv_agg's binary_logloss: 0.243456 + 0.000813292
[167]	cv_agg's binary_logloss: 0.243456 + 0.000814457
[168]	cv_agg's binary_logloss: 0.243452 + 0.000813959
[169]	cv_agg's binary_logloss: 0.243451 + 0.000814753
[170]	cv_agg's binary_logloss: 0.24345 + 0.000814881
[171]	cv_agg's binary_logloss: 0.243447 + 0.000814858
[172]	cv_agg's binary_logloss: 0.243446 + 0.000814546
[173]	cv_agg's binary_logloss: 0.243444 + 0.000811353
[174]	cv_agg's binary_logloss: 0.243442 + 0.000811042
[175]	cv_agg's binary_logloss: 

In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline

In [10]:
f = open('gridsearch_result.save', 'wb')
cPickle.dump(results, f, protocol=cPickle.HIGHEST_PROTOCOL)
f.close()
# steps = [('scaler', StandardScaler()), ('elasticnet', ElasticNet())]

In [11]:
# pipeline = Pipeline(steps)

In [12]:
# parameters = {'elasticnet__l1_ratio':np.linspace(0, 1, 30)}

In [13]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# model = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=5)

In [15]:
# model.fit(X_train, y_train)

In [16]:
# y_pred = model.predict(X_test)